In [ ]:
#credit to python10pm https://www.kaggle.com/python10pm/nfl-training-xgboost-model#Model-Training



# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import matplotlib.pyplot as plt
from kaggle.competitions import nflrush

from sklearn import preprocessing
from sklearn.model_selection import KFold
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV
from sklearn.model_selection import StratifiedShuffleSplit

from xgboost import XGBRegressor
import scipy


# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.


Importing the data


In [ ]:
train_df=pd.read_csv('/kaggle/input/nfl-big-data-bowl-2020/train.csv', low_memory=False)

> **Pre Proccesing **

In [ ]:
# columns to drop after preprocesing
drop_columns = [\
"GameId", 
"PlayId",  
"JerseyNumber", 
"Season",
"NflId",
"OffensePersonnel", 
"DisplayName",
"GameClock",
"TimeHandoff", 
"TimeSnap", 
"PlayerBirthDate", 
"PlayerCollegeName",
"Stadium",
"WindDirection",
"WindSpeed",
"Week"


  ]

Transform some subjective columns into numerical values

In [ ]:
df=train_df.copy(deep=True)
df["Team"]=df["Team"].apply(lambda x: 0 if x=="away" else 1)
df["PlayDirection"]=df["PlayDirection"].apply(lambda x: 0 if x=="left" else 1)
df["PlayerHeight"] = df["PlayerHeight"].apply(lambda x: (float(x.split("-")[0])+float(x.split("-")[1])/12))
print(df["PlayerHeight"])
#list of outdoor stadiums
typesofstadiums=[\
"Outdoor",
"Outdoors",
"Open",
"Oudoor",
"Outddors",
"Ourdoor",
"Outdor",
"Outside",
"Retr. Roof-Open",
"Outdoor Retr Roof-Open",
"Retr. Roof - Open",
"Indoor, Open Roof",
"Domed, Open",
"Heinz Field",
"Bowl",
"Retractable Roof",
"Cloudy"]
df.fillna({"StadiumType":"Unknown"}, inplace = True)
df["StadiumType"]=df["StadiumType"].apply(lambda x: 1  if x in typesofstadiums else 0)

#list of artificial turfs
turftype=[\
'Turf'
'Field Turf', 
'Artificial', 
'FieldTurf',
'UBU Speed Series-S5-M', 
'A-Turf Titan', 
'UBU Sports Speed S5-M',
'FieldTurf360', 
'DD GrassMaster', 
'Twenty-Four/Seven Turf', 
'SISGrass',
'FieldTurf 360',
'Artifical', 
'Field turf']
df.fillna({"Turf":"Unknown"}, inplace=True)
df["Turf"]=df["Turf"].apply(lambda x: -1 if x in turftype else 0)


arr = [[float(s) for s in t.split(":")] for t in df["GameClock"]]
df["QuarterTime"] = pd.Series([a[0]+a[1]/60.0 for a in arr]) #make the quarter time into a numerical value


    

Columns still to be preprocessed

In [ ]:
cols=[]

for c in df.columns:
    if c not in drop_columns+["Turf", "StadiumType","PlayDirection" ,"PlayerHeight" ,"Team","QuarterTime","Yards"]:
        cols.append(c)
print(cols)



In [ ]:
label_dict = {}
for c in cols:
    if c == "DefensePersonnel": #credit goes to hukada222
        arr = [[int(s[0]) for s in t.split(", ")] for t in df["DefensePersonnel"]]
        df["DL"] = np.array([a[0] for a in arr])
        df["LB"] = np.array([a[1] for a in arr])
        df["DB"] = np.array([a[2] for a in arr])

   
    elif df[c].dtype=='object': 
        lbl = preprocessing.LabelEncoder()
        lbl.fit(list(df[c].values))
        label_dict[c] = lbl
        df[c] = lbl.transform(list(df[c].values))
drop_columns.append("DefensePersonnel")
print(label_dict) 

Time to reshape this data

In [ ]:
    df.sort_values(by = ["GameId", "PlayId"]).reset_index() 
    df.drop(list(drop_columns), inplace = True, axis = 1)
    print(df.columns)
   # print(df)

In [ ]:

columns = list(df.columns)
columns.remove("Yards")

X_train = df[columns]
y_train = df["Yards"]

y = y_train.values
target = y[np.arange(0, len(X_train), 22)]

standard_deviation = np.std(target)

print(X_train)
print(y_train)

In [ ]:
model = XGBRegressor()
model.fit(X_train, y_train)


Send Submission File

In [ ]:
def preprocess(df):
    drop_columns = [\
        "GameId", 
        "PlayId",  
        "JerseyNumber", 
        "Season",
        "NflId",
        "OffensePersonnel", 
        "DisplayName",
        "GameClock",
        "TimeHandoff", 
        "TimeSnap", 
        "PlayerBirthDate", 
        "PlayerCollegeName",
        "Stadium",
        "WindDirection",
        "WindSpeed",
        "Week"


      ]
    
    df["Team"]=df["Team"].apply(lambda x: 0 if x=="away" else 1)
    df["PlayDirection"]=df["PlayDirection"].apply(lambda x: 0 if x=="left" else 1)
    df["PlayerHeight"] = df["PlayerHeight"].apply(lambda x: (float(x.split("-")[0])+float(x.split("-")[1])/12))
   
    #list of outdoor stadiums
    typesofstadiums=[\
    "Outdoor",
    "Outdoors",
    "Open",
    "Oudoor",
    "Outddors",
    "Ourdoor",
    "Outdor",
    "Outside",
    "Retr. Roof-Open",
    "Outdoor Retr Roof-Open",
    "Retr. Roof - Open",
    "Indoor, Open Roof",
    "Domed, Open",
    "Heinz Field",
    "Bowl",
    "Retractable Roof",
    "Cloudy"]
    df.fillna({"StadiumType":"Unknown"}, inplace = True)
    df["StadiumType"]=df["StadiumType"].apply(lambda x: 1  if x in typesofstadiums else 0)

    #list of artificial turfs
    turftype=[\
    'Turf'
    'Field Turf', 
    'Artificial', 
    'FieldTurf',
    'UBU Speed Series-S5-M', 
    'A-Turf Titan', 
    'UBU Sports Speed S5-M',
    'FieldTurf360', 
    'DD GrassMaster', 
    'Twenty-Four/Seven Turf', 
    'SISGrass',
    'FieldTurf 360',
    'Artifical', 
    'Field turf']
    df.fillna({"Turf":"Unknown"}, inplace=True)
    df["Turf"]=df["Turf"].apply(lambda x: -1 if x in turftype else 0)


    arr = [[float(s) for s in t.split(":")] for t in df["GameClock"]]
    df["QuarterTime"] = pd.Series([a[0]+a[1]/60.0 for a in arr]) #make the quarter time into a numerical value
    cols=[]

    for c in df.columns:
        if c not in drop_columns+["Turf", "StadiumType","PlayDirection" ,"PlayerHeight" ,"Team","QuarterTime","Yards"]:
            cols.append(c)
    
    for c in cols:
        if c == "DefensePersonnel": #credit goes to hukada222
            arr = [[int(s[0]) for s in t.split(", ")] for t in df["DefensePersonnel"]]
            df["DL"] = np.array([a[0] for a in arr])
            df["LB"] = np.array([a[1] for a in arr])
            df["DB"] = np.array([a[2] for a in arr])


        elif c in label_dict and df[c].dtype=='object' and not pd.isnull(test_df[c]).any(): 
            try:
                df[c] = lbl_dict[c].transform(list(df[c].values))
            except:
                df[c] = [np.nan for i in len(df[c])]
    
            
    drop_columns.append("DefensePersonnel")
    df.sort_values(by = ["GameId", "PlayId"]).reset_index() 
    df.drop(list(drop_columns), inplace = True, axis = 1)
    
    df.fillna(-999, inplace = True)
    columns = list(df.columns)
    columns.remove("Yards")

    X_train = df[columns]
    return X_train


import tqdm
from kaggle.competitions import nflrush
env = nflrush.make_env()


for (test_df, sample_prediction_df) in tqdm.tqdm(env.iter_test()):
    try:
        test_df_processed = preprocess(test_df)
        y_pred_p = model.predict(test_df_processed)
        y_pred_first = y_pred_p[0]
        
    except:
        test_df
        y_pred_first = 1
        
    pred_df = np.zeros((1, 199))

    for x in range(len(pred_df[0])):
        current_cdf = scipy.stats.norm(loc = y_pred_first, scale = standard_deviation).cdf(x-99)
        pred_df[0][x] = current_cdf


    final_pred_df = pd.DataFrame(data=pred_df, columns=sample_prediction_df.columns)
    env.predict(final_pred_df)
    
env.write_submission_file()














